In [ ]:
# uninstall all tensorflow packages
! pip list --format=freeze | grep '^tensorflow' | cut -d= -f1 | xargs -n1 pip uninstall -y

# install CPU-only version of tensorflow; used for image preprocessing
! pip install einops tensorflow-cpu torchvision


Found existing installation: tensorflow 2.17.0
Uninstalling tensorflow-2.17.0:
  Successfully uninstalled tensorflow-2.17.0
Found existing installation: tensorflow-datasets 4.9.6
Uninstalling tensorflow-datasets-4.9.6:
  Successfully uninstalled tensorflow-datasets-4.9.6
Found existing installation: tensorflow-hub 0.16.1
Uninstalling tensorflow-hub-0.16.1:
  Successfully uninstalled tensorflow-hub-0.16.1
Found existing installation: tensorflow-io-gcs-filesystem 0.37.1
Uninstalling tensorflow-io-gcs-filesystem-0.37.1:
  Successfully uninstalled tensorflow-io-gcs-filesystem-0.37.1
Found existing installation: tensorflow-metadata 1.15.0
Uninstalling tensorflow-metadata-1.15.0:
  Successfully uninstalled tensorflow-metadata-1.15.0
Found existing installation: tensorflow-probability 0.24.0
Uninstalling tensorflow-probability-0.24.0:
  Successfully uninstalled tensorflow-probability-0.24.0
  Using cached tensorflow_io_gcs_filesystem-0.37.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_

In [ ]:
from transformers import AutoModelForCausalLM, AutoProcessor, GenerationConfig
from PIL import Image
import requests

# load the processor
processor = AutoProcessor.from_pretrained(
    'allenai/MolmoE-1B-0924',
    trust_remote_code=True,
    torch_dtype='auto',
    device_map='auto'
)

# load the model
model = AutoModelForCausalLM.from_pretrained(
    'allenai/MolmoE-1B-0924',
    trust_remote_code=True,
    torch_dtype='auto',
    device_map='auto'
)

# process the image and text
inputs = processor.process(
    images=[Image.open(requests.get("https://lakshonline.com/wp-content/uploads/2024/09/image-2.png", stream=True).raw)],
    text="Describe this image."
)

# move inputs to the correct device and make a batch of size 1
inputs = {k: v.to(model.device).unsqueeze(0) for k, v in inputs.items()}

# generate output; maximum 200 new tokens; stop generation when <|endoftext|> is generated
output = model.generate_from_batch(
    inputs,
    GenerationConfig(max_new_tokens=200, stop_strings="<|endoftext|>"),
    tokenizer=processor.tokenizer
)

# only get generated tokens; decode them to text
generated_tokens = output[0,inputs['input_ids'].size(1):]
generated_text = processor.tokenizer.decode(generated_tokens, skip_special_tokens=True)

# print the generated text
print(generated_text)

# >>> This photograph captures a small black puppy, likely a Labrador or a similar breed,
#     sitting attentively on a weathered wooden deck. The deck, composed of three...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPTNeoXTokenizer'. 
The class this function is called from is 'GPT2TokenizerFast'.


 The image is a detailed, computer-generated illustration that merges two distinct time periods into one scene. On the left side, it depicts a nostalgic, old-fashioned cityscape reminiscent of the late 1800s or early 1900s. This side features a cobblestone street with a vintage car parked along the curb, a bicycle leaning against a lamppost, and a teapot sitting on the sidewalk. The buildings lining the street are three to four stories tall, with lit windows and a brick sidewalk. The sky above is a mix of blue and pink hues, suggesting either sunrise or sunset.

In stark contrast, the right side of the image transitions into a futuristic, cyberpunk-inspired vision of the same city. This side showcases towering skyscrapers adorned with neon lights and illuminated windows, creating a stark juxtaposition against the older architecture. The sky here is a deep blue, dotted with stars, enhancing the sci-
